In [3]:
import matplotlib.pyplot as plt
import numpy as np 
from glob import glob
import pandas as pd
import emcee
from astropy import units as u
from astropy import constants as const
from zipfile import ZipFile

In [2]:
#grab anything that contains '.fits'
files = glob('*.fits')
mcfiles = glob('*emcee.h5')
print(files)
# len(files)
print(mcfiles)

['hd269879_122722.fits', 'hd269857_122722.fits', 'hd27005_122722.fits', 'hd269723_122722.fits', 'hd269662_122622.fits', 'hd269070_122622.fits', 'sk_69_99_122622.fits', 'hd269697_122722.fits', 'hd269902_122722.fits', 'w60_d17_122622.fits', 'hd269331_122622.fits', 'hd269661_122622.fits', 'hd268727_010323.fits', 'hd269762_122722.fits', 'hd269953_122722.fits', 'hd33579_122622.fits', 'hv2450_122622.fits', 'hv883_010323.fits', 'hd269604_122622.fits', 'hd268819_122622.fits', 'j05344_122722.fits', 'cd_69_310_122622.fits', 'sp77_31_16_122622.fits', 'cpd-69-496_122722.fits', 'hd269781_122722.fits', 'hd268687_122622.fits', 'sp77_48_6_122622.fits', 'hd268828_010323.fits', 'hd269651_122622.fits', 'hd269982_122722.fits', 'sk_69_148_122622.fits', 'hd268949_010323.fits', 'hd268946_122622.fits', 'hd269840_122722.fits', 'rm177_010323.fits', 'hd269110_122622.fits', 'hd269787_122722.fits', 'hd269807_122722.fits', 'hd268971_122622.fits', 'hd268865_010323.fits']
['sp77_48_6_122622.fits_emcee.h5', 'hd269331_

In [5]:
mcfiles_sorted = [] # create empty list

# iterate through each value in files and if they start the same add it to the 
# new list so that mcfiles is ordered the same as files
for j in range(len(files)): 
    for i in range(len(mcfiles)):
        mcfilehead=mcfiles[i][:-9]
        if files[j]==mcfilehead:
            mcfiles_sorted.append(mcfiles[i])
            
print('mcfiles not sorted:\n' + str(mcfiles))
print('\nmcfiles sorted:\n' + str(mcfiles_sorted))
print('\nfiles sorted:\n' + str(files))
len(files)

mcfiles not sorted:
['sp77_48_6_122622.fits_emcee.h5', 'hd269331_122622.fits_emcee.h5', 'hd269787_122722.fits_emcee.h5', 'hd33579_122622.fits_emcee.h5', 'hv883_010323.fits_emcee.h5', 'cpd-69-496_122722.fits_emcee.h5', 'hd269902_122722.fits_emcee.h5', 'hd269651_122622.fits_emcee.h5', 'hd269110_122622.fits_emcee.h5', 'hd269982_122722.fits_emcee.h5', 'hd269070_122622.fits_emcee.h5', 'hd268819_122622.fits_emcee.h5', 'hd269807_122722.fits_emcee.h5', 'cd_69_310_122622.fits_emcee.h5', 'hd268865_010323.fits_emcee.h5', 'sk_69_99_122622.fits_emcee.h5', 'hd269662_122622.fits_emcee.h5', 'hd27005_122722.fits_emcee.h5', 'hd269762_122722.fits_emcee.h5', 'hd269723_122722.fits_emcee.h5', 'sk_69_148_122622.fits_emcee.h5', 'hd269661_122622.fits_emcee.h5', 'hd269857_122722.fits_emcee.h5', 'hd269840_122722.fits_emcee.h5', 'w60_d17_122622.fits_emcee.h5', 'hd268687_122622.fits_emcee.h5', 'sp77_31_16_122622.fits_emcee.h5', 'hd269953_122722.fits_emcee.h5', 'hd268828_010323.fits_emcee.h5', 'hv2450_122622.fits_e

40

In [8]:
def loademcee(i):
    """
    reads the h5 file and generates theta values where theta is [t_eff, log_g, rv, ebv, radius]
    """
    # load the file
    reader = emcee.backends.HDFBackend('/Users/kchen/FYPS/' + mcfiles_sorted[i], name='rv_noise_prior')
    #discard first 1000 points where it's converging
    if i==13:
        samples = reader.get_chain(flat=True, discard=5100) # weird
    else:
        samples = reader.get_chain(flat=True, discard=1000)
    
    # pick 32 values between 0-128000 from the sample index and theta
    rand_ind=np.random.randint(0,127999, size=32)
    for i in rand_ind:
        theta = samples[i] #for each theta can plot each stsynphot model as trnsparent/thin posterior predictive plot
    return samples

In [9]:
# read in csv
df = pd.read_csv('/Users/kchen/Downloads/ysg_new.csv')
# add spec lum columns
df['lum_mc'] = None
df['lum_upper'] = None
df['lum_lower'] = None

# add spec inverse of the flux-mean gravity (fancy L) columns
df['ifwg_mc'] = None
df['ifwg_upper'] = None
df['ifwg_lower'] = None

# add column
df['binary'] = None

# see what it looks like
print(df.head())

   index CommonName  teff_mc  teff_upper  teff_lower  logg_mc  logg_upper  \
0      0  HD 269879      NaN         NaN         NaN      NaN         NaN   
1      1  HD 269857      NaN         NaN         NaN      NaN         NaN   
2      2  HD 270050      NaN         NaN         NaN      NaN         NaN   
3      3  HD 269723      NaN         NaN         NaN      NaN         NaN   
4      4  HD 269662      NaN         NaN         NaN      NaN         NaN   

   logg_lower  radius_mc  radius_upper  ...  halpha_emission   FYPS  \
0         NaN        NaN           NaN  ...             True  False   
1         NaN        NaN           NaN  ...             True  False   
2         NaN        NaN           NaN  ...            False  False   
3         NaN        NaN           NaN  ...             True   True   
4         NaN        NaN           NaN  ...             True  False   

             source_id  lum_mc  lum_upper  lum_lower ifwg_mc ifwg_upper  \
0  4660276052906311296    None     

In [10]:
def editcsv():
    """
    Edits the ysg csv by adding in values and columns based on values from emcee runs
    """
    
    for i in range(len(mcfiles_sorted)):
        samples = loademcee(i)
        x = samples[:,0] #teff
        y = samples[:,1] #logg
        z = samples[:,4] #radius
        
        # calculate percentiles
        teff = np.percentile(x, [16, 50, 84]) # array of 16th, 50th, and 84th, percentile for teff values
        teff_median = teff[1] # median teff
        
        logg = np.percentile(y, [16, 50, 84]) # array of 16th, 50th, and 84th, percentile for logg values
        logg_median = logg[1] # median logg
        
        rad = np.percentile(z, [16, 50, 84]) # array of 16th, 50th, and 84th, percentile for radius values
        rad_median = rad[1] # median radius
        
        # spec luminosity values on hr diagram
        L = (((4*np.pi))*(z**2 * u.R_sun**2)*const.sigma_sb*(x**4 * u.K**4)).to(u.L_sun) # Stefan-Boltzmann equation to get L
        sb = np.log10(L.value) # log(L/L_odot)
        
        sbp = np.percentile(sb, [16, 50, 84]) # percentiles of L
        sb_median = sbp[1]
        
        #compute inverse of the flux-mean gravity (fancy L)
        fancyL =  4*np.log10(x)-y
        fancyLsun = 4*np.log10(5772)-np.log10(27400)
        
        fancyLp = np.percentile(fancyL, [16, 50, 84]) - fancyLsun # percentiles of fancy L
        l_median = fancyLp[1] # median fancy L
    
        # edit teff columns
        df.loc[i, 'teff_mc'] = np.log10(teff_median)
        df.loc[i, 'teff_lower'] = np.log10(teff[0])
        df.loc[i, 'teff_upper'] = np.log10(teff[2])
        
        # edit logg columns
        df.loc[i, 'logg_mc'] = logg_median
        df.loc[i, 'logg_lower'] = logg[0]
        df.loc[i, 'logg_upper'] = logg[2]
        
        # edit radius columns
        df.loc[i, 'radius_mc'] = rad_median
        df.loc[i, 'radius_lower'] = rad[0]
        df.loc[i, 'radius_upper'] = rad[2]
        
        # edit spec luminosity columns
        df.loc[i, 'lum_mc'] = sb_median
        df.loc[i, 'lum_lower'] = sbp[0]
        df.loc[i, 'lum_upper'] = sbp[2]
        
        # edit inverse of the flux-mean gravity (fancy L) columns
        df.loc[i, 'ifwg_mc'] = l_median
        df.loc[i, 'ifwg_lower'] = fancyLp[0]
        df.loc[i, 'ifwg_upper'] = fancyLp[2]

In [11]:
editcsv()

In [151]:
const.sigma_sb

<<class 'astropy.constants.codata2018.CODATA2018'> name='Stefan-Boltzmann constant' value=5.6703744191844314e-08 uncertainty=0.0 unit='W / (K4 m2)' reference='CODATA 2018'>

In [12]:
df.tail()

,index,CommonName,teff_mc,teff_upper,teff_lower,logg_mc,logg_upper,logg_lower,radius_mc,radius_upper,...,halpha_emission,FYPS,source_id,lum_mc,lum_upper,lum_lower,ifwg_mc,ifwg_upper,ifwg_lower,binary
35,35,HD 269110,3.708882,3.709401,3.708376,0.774152,0.844383,0.701712,375.429450,376.577232,...,True,False,4652214880388720000,4.939264,4.940154,4.938395,3.453659,3.527754,3.382029,None
36,36,HD 269787,3.877494,3.877796,3.877195,2.998588,2.999653,2.996302,165.592685,165.781121,...,True,True,4660246606598419968,4.90275,4.903185,4.90232,1.904095,1.90645,1.902426,None
37,37,HD 269807,3.864868,3.865317,3.864410,2.987470,2.996883,2.967997,156.873423,157.136522,...,False,False,4660245300928732160,4.805127,4.806169,4.804393,1.86472,1.883325,1.855895,None
38,38,HD 268971,4.070003,4.070029,4.069944,2.000480,2.001289,2.000121,78.985715,79.080517,...,False,False,4655053059131622016,5.029738,5.030785,5.028644,3.67189,3.672325,3.67106,None
39,39,HD 268865,3.732994,3.733324,3.732672,1.886304,1.968467,1.796306,228.601557,228.993601,...,False,False,4661327323506601344,4.60482,4.605548,4.604112,2.43828,2.528556,2.355663,None


In [182]:
df.to_csv('/Users/kchen/Downloads/ysg.csv')

In [1]:
has_binary = [0,2,4,6,9,11,16,17,18,20,22,26,27,28,30,31,34,35,37,39]
len(has_binary)

20

In [13]:
# list for if it has binary
has_binary = [0,2,4,6,9,11,16,17,18,20,22,26,27,28,30,31,34,35,37,39]
# list for maybe binary
maybe_binary = [1,3,5,7,10,14,19,21,23,25,29,32,33,36,38]

def editcsv1():
    """
    Edits the ysg csv by adding in values for if the star is in a binary system. 0 if no, 1 if yes, 2 if maybe.
    """
    
    for i in range(len(mcfiles_sorted)):
        if i in has_binary:
            df.loc[i, 'binary'] = 1
        elif i in maybe_binary:
            df.loc[i, 'binary'] = 2
        else: # no binary
            df.loc[i, 'binary'] = 0

In [14]:
editcsv1()
df.tail()

,index,CommonName,teff_mc,teff_upper,teff_lower,logg_mc,logg_upper,logg_lower,radius_mc,radius_upper,...,halpha_emission,FYPS,source_id,lum_mc,lum_upper,lum_lower,ifwg_mc,ifwg_upper,ifwg_lower,binary
35,35,HD 269110,3.708882,3.709401,3.708376,0.774152,0.844383,0.701712,375.429450,376.577232,...,True,False,4652214880388720000,4.939264,4.940154,4.938395,3.453659,3.527754,3.382029,1
36,36,HD 269787,3.877494,3.877796,3.877195,2.998588,2.999653,2.996302,165.592685,165.781121,...,True,True,4660246606598419968,4.90275,4.903185,4.90232,1.904095,1.90645,1.902426,2
37,37,HD 269807,3.864868,3.865317,3.864410,2.987470,2.996883,2.967997,156.873423,157.136522,...,False,False,4660245300928732160,4.805127,4.806169,4.804393,1.86472,1.883325,1.855895,1
38,38,HD 268971,4.070003,4.070029,4.069944,2.000480,2.001289,2.000121,78.985715,79.080517,...,False,False,4655053059131622016,5.029738,5.030785,5.028644,3.67189,3.672325,3.67106,2
39,39,HD 268865,3.732994,3.733324,3.732672,1.886304,1.968467,1.796306,228.601557,228.993601,...,False,False,4661327323506601344,4.60482,4.605548,4.604112,2.43828,2.528556,2.355663,1


In [15]:
df.to_csv('/Users/kchen/Downloads/ysg_new.csv')